# Koltsov3 Parameter Space Exploration

Systematically explore the Koltsov3 generator (perm_type=2) across different coset groups.

## Search Space
- **k values**: 0, 1, 2, ... (where k+3 < n)
- **n values**: 4 to 30
- **coset types**: 5 groups (run separately)

## Coset Groups
1. `different` - 2Different, 3Different, 4Different
2. `then` - Binary0then1, 0then1then2, etc.
3. `coincide` - 2Coincide, 3Coincide, etc.
4. `repeats` - Binary01Repeats, 012Repeats, etc.
5. `full_graph` - No coset (max_n=12)

## Cell 1: Imports and Setup

In [1]:
try:
    import numba
    # Use for CPU,GPU Kaggle machines:
    print('Install CayleyPy without dependencies (for Kaggle CPU,GPU):'); print()
    !pip install git+https://github.com/cayleypy/cayleypy --no-deps -q
except:
    print('Install CayleyPy with dependencies (for Kaggle TPU):'); print()
    # Use for TPU kaggle machines - since no numba
    !pip install git+https://github.com/cayleypy/cayleypy -q

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
import os
from datetime import datetime
from tqdm.auto import tqdm
from cayleypy import CayleyGraph, PermutationGroups

print("Imports successful!")

Install CayleyPy without dependencies (for Kaggle CPU,GPU):



/usr/local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Imports successful!


## Cell 2: Configuration

In [2]:
PERM_TYPE = 2
MIN_N = 4
MAX_N = 30
OUTPUT_DIR = "results_perm2"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Config: perm_type={PERM_TYPE}, n=[{MIN_N}, {MAX_N}]")
print(f"Output directory: {OUTPUT_DIR}")

Config: perm_type=2, n=[4, 30]
Output directory: results_perm2


## Cell 3: Coset Group Definitions

In [6]:
COSET_GROUPS = {
    # =========================================================================
    # FULL GRAPH - No coset, explores entire permutation space
    # =========================================================================
    "full_graph": {
        # n=6: None (uses identity, explores all 6! = 720 permutations)
        "FullGraph": lambda n: None,
    },
    
    # =========================================================================
    # DIFFERENT - First D-1 elements are distinct, rest are all the same
    # Pattern: [0, 1, ..., D-2, D-1, D-1, D-1, ...]
    # =========================================================================
    "different": {
        # n=6: [0, 1, 1, 1, 1, 1]
        "2Different": lambda n: list(range(1)) + [1]*(n-1) if n >= 2 else None,
        # n=6: [0, 1, 2, 2, 2, 2]
        "3Different": lambda n: list(range(2)) + [2]*(n-2) if n >= 3 else None,
        # n=6: [0, 1, 2, 3, 3, 3]
        "4Different": lambda n: list(range(3)) + [3]*(n-3) if n >= 4 else None,
    },
    
    # =========================================================================
    # THEN - Blocks of consecutive same values
    # Pattern: [0,0,..., 1,1,..., 2,2,...] (divided into equal-ish parts)
    # =========================================================================
    "then": {
        # n=6: [0, 0, 0, 1, 1, 1]
        "Binary0then1": lambda n: [0]*(n//2) + [1]*(n - n//2),
        # n=6: [0, 0, 1, 1, 2, 2]
        "0then1then2": lambda n: [0]*(n//3) + [1]*(n//3) + [2]*(n - 2*(n//3)),
        # n=8: [0, 0, 1, 1, 2, 2, 3, 3]
        "0then1then2then3": lambda n: [0]*(n//4) + [1]*(n//4) + [2]*(n//4) + [3]*(n - 3*(n//4)),
        # n=10: [0, 0, 1, 1, 2, 2, 3, 3, 4, 4]
        "0then1then2then3then4": lambda n: [0]*(n//5) + [1]*(n//5) + [2]*(n//5) + [3]*(n//5) + [4]*(n - 4*(n//5)),
    },
    
    # =========================================================================
    # COINCIDE - Sequential values, but last C elements are all the same
    # Pattern: [0, 1, 2, ..., n-C-1, n-C, n-C, ..., n-C]
    # =========================================================================
    "coincide": {
        # n=6: [0, 1, 2, 3, 4, 4]
        "2Coincide": lambda n: list(range(n-2)) + [n-2]*2 if n > 2 else None,
        # n=6: [0, 1, 2, 3, 3, 3]
        "3Coincide": lambda n: list(range(n-3)) + [n-3]*3 if n > 3 else None,
        # n=6: [0, 1, 2, 2, 2, 2]
        "4Coincide": lambda n: list(range(n-4)) + [n-4]*4 if n > 4 else None,
        # n=6: [0, 1, 1, 1, 1, 1]
        "5Coincide": lambda n: list(range(n-5)) + [n-5]*5 if n > 5 else None,
        # n=7: [0, 1, 1, 1, 1, 1, 1]
        "6Coincide": lambda n: list(range(n-6)) + [n-6]*6 if n > 6 else None,
    },
    
    # =========================================================================
    # REPEATS - Repeating pattern of a short block
    # Pattern: [block] * (n // len(block)) + partial
    # =========================================================================
    "repeats": {
        # n=6: [0, 1, 0, 1, 0, 1]
        # n=7: [0, 1, 0, 1, 0, 1, 0]
        "Binary01Repeats": lambda n: [0,1]*(n//2) + [0]*(n - 2*(n//2)),
        # n=6: [0, 1, 0, 1, 0, 1]
        # n=7: [0, 1, 0, 1, 0, 1, 1]
        "Binary01Repeats_1": lambda n: [0,1]*(n//2) + [1]*(n - 2*(n//2)),
        # n=6: [0, 1, 2, 0, 1, 2]
        # n=7: [0, 1, 2, 0, 1, 2, 0]
        "012Repeats": lambda n: [0,1,2]*(n//3) + [0,1,2][:(n%3)],
        # n=6: [0, 1, 1, 0, 1, 1]
        # n=7: [0, 1, 1, 0, 1, 1, 0]
        "011Repeats": lambda n: [0,1,1]*(n//3) + [0,1,1][:(n%3)],
    },
}

# Print summary with examples
print("Coset Groups Summary:")
print("=" * 60)
for group_name, cosets in COSET_GROUPS.items():
    print(f"\n{group_name.upper()}:")
    for coset_name, func in cosets.items():
        example = func(6) if func(6) is not None else "None (full graph)"
        print(f"  {coset_name}: n=6 -> {example}")

Coset Groups Summary:

FULL_GRAPH:
  FullGraph: n=6 -> None (full graph)

DIFFERENT:
  2Different: n=6 -> [0, 1, 1, 1, 1, 1]
  3Different: n=6 -> [0, 1, 2, 2, 2, 2]
  4Different: n=6 -> [0, 1, 2, 3, 3, 3]

THEN:
  Binary0then1: n=6 -> [0, 0, 0, 1, 1, 1]
  0then1then2: n=6 -> [0, 0, 1, 1, 2, 2]
  0then1then2then3: n=6 -> [0, 1, 2, 3, 3, 3]
  0then1then2then3then4: n=6 -> [0, 1, 2, 3, 4, 4]

COINCIDE:
  2Coincide: n=6 -> [0, 1, 2, 3, 4, 4]
  3Coincide: n=6 -> [0, 1, 2, 3, 3, 3]
  4Coincide: n=6 -> [0, 1, 2, 2, 2, 2]
  5Coincide: n=6 -> [0, 1, 1, 1, 1, 1]
  6Coincide: n=6 -> None (full graph)

REPEATS:
  Binary01Repeats: n=6 -> [0, 1, 0, 1, 0, 1]
  Binary01Repeats_1: n=6 -> [0, 1, 0, 1, 0, 1]
  012Repeats: n=6 -> [0, 1, 2, 0, 1, 2]
  011Repeats: n=6 -> [0, 1, 1, 0, 1, 1]


## Cell 4: Helper Functions

In [7]:
def is_valid_central(central):
    """Checks if central has >1 unique value."""
    return central is not None and len(np.unique(central)) > 1

def run_single_experiment(n, k, coset_name, coset_func):
    """Runs BFS for given parameters, returns BfsResult or None."""
    try:
        defn = PermutationGroups.koltsov3(n, perm_type=PERM_TYPE, k=k)

        central = coset_func(n)
        if coset_name != "FullGraph":
            if not is_valid_central(central):
                return None
            defn = defn.with_central_state(central)

        graph = CayleyGraph(defn)
        return graph.bfs(return_all_edges=False, return_all_hashes=False)
    except Exception as e:
        print(f"Error: {coset_name}, k={k}, n={n}: {e}")
        return None

def get_group_dir(group_name):
    """Get the output directory for a group, creating it if needed."""
    group_dir = f"{OUTPUT_DIR}/{group_name}"
    os.makedirs(group_dir, exist_ok=True)
    return group_dir

def get_computed_combinations(group_name):
    """Return set of (coset, k, n) tuples already computed."""
    group_dir = get_group_dir(group_name)
    csv_path = f"{group_dir}/data.csv"

    if not os.path.exists(csv_path):
        return set()

    df = pd.read_csv(csv_path)
    return set(zip(df['coset'], df['k'], df['n']))

def run_group(group_name, cosets, min_n=MIN_N, max_n=MAX_N, k_range=None, coset_filter=None, skip_computed=True):
    """
    Run experiments for a group of cosets.

    Args:
        group_name: Name of coset group
        cosets: Dictionary of coset functions
        min_n: Minimum n value (default: MIN_N)
        max_n: Maximum n value (default: MAX_N)
        k_range: Optional tuple (k_min, k_max); if None, k ranges 0 to max_n-4
        coset_filter: Filter which cosets to run:
            - None: run all cosets in the group (default)
            - str: run only that coset (e.g., "3Different")
            - list: run only those cosets (e.g., ["2Different", "4Different"])
        skip_computed: If True, skip (coset, k, n) combinations already in CSV

    Returns:
        Dictionary with results for new computations only
    """
    computed = get_computed_combinations(group_name) if skip_computed else set()

    # Filter cosets based on coset_filter
    if coset_filter is None:
        filtered_cosets = cosets
    elif isinstance(coset_filter, str):
        if coset_filter not in cosets:
            raise ValueError(f"Coset '{coset_filter}' not found in group. Available: {list(cosets.keys())}")
        filtered_cosets = {coset_filter: cosets[coset_filter]}
    else:  # list
        filtered_cosets = {k: v for k, v in cosets.items() if k in coset_filter}
        missing = set(coset_filter) - set(filtered_cosets.keys())
        if missing:
            raise ValueError(f"Cosets not found: {missing}. Available: {list(cosets.keys())}")

    if k_range is None:
        k_values = list(range(max_n))
    else:
        k_values = list(range(k_range[0], k_range[1] + 1))

    results = {
        "metadata": {
            "perm_type": PERM_TYPE,
            "group": group_name,
            "timestamp": datetime.now().isoformat(),
            "n_range": [min_n, max_n],
            "k_range": list(k_range) if k_range else None,
            "coset_filter": coset_filter
        },
        "results": {}
    }

    total_new = 0

    for coset_name, coset_func in filtered_cosets.items():
        skipped = 0
        computed_count = 0
        results["results"][coset_name] = {}

        # Create progress bar for this coset
        pbar = tqdm(total=len(k_values) * (max_n - min_n + 1), 
                    desc=f"{coset_name}", 
                    leave=True)

        for k in k_values:
            k_key = f"k={k}"
            results["results"][coset_name][k_key] = {}

            for n in range(min_n, max_n + 1):
                pbar.set_postfix({"k": k, "n": n})
                pbar.update(1)
                
                if k + 3 >= n:
                    continue

                # Skip if already computed
                if (coset_name, k, n) in computed:
                    skipped += 1
                    continue

                result = run_single_experiment(n, k, coset_name, coset_func)
                if result is not None:
                    results["results"][coset_name][k_key][f"n={n}"] = {
                        "diameter": result.diameter(),
                        "growth": result.layer_sizes,
                        "last_layer_size": len(result.last_layer())
                    }
                    computed_count += 1

        pbar.close()
        total_new += computed_count
        print(f"  {coset_name}: Skipped {skipped} cached, computed {computed_count} new")

    print(f"Completed {group_name} ({total_new} new results)")
    return results

def save_results(group_name, results, cosets, append=True):
    """
    Save results to CSV (growth and central stored as JSON strings).

    Args:
        group_name: Name of coset group
        results: Results dictionary from run_group()
        cosets: Dictionary of coset functions (to compute central states)
        append: If True, append to existing CSV; if False, overwrite

    Returns:
        DataFrame with all results (existing + new)
    """
    group_dir = get_group_dir(group_name)
    csv_path = f"{group_dir}/data.csv"

    # Build DataFrame from new results
    rows = []
    for coset, k_data in results["results"].items():
        for k_key, n_data in k_data.items():
            k_val = int(k_key.split("=")[1])
            for n_key, metrics in n_data.items():
                n_val = int(n_key.split("=")[1])
                rows.append({
                    "coset": coset,
                    "k": k_val,
                    "n": n_val,
                    "diameter": metrics["diameter"],
                    "last_layer_size": metrics["last_layer_size"],
                    "total_states": sum(metrics["growth"]),
                    "growth": json.dumps(metrics["growth"])
                })

    df_new = pd.DataFrame(rows)

    if append and os.path.exists(csv_path):
        df_existing = pd.read_csv(csv_path)
        df = pd.concat([df_existing, df_new], ignore_index=True)
        # Remove duplicates, keeping last (new) values
        df = df.drop_duplicates(subset=['coset', 'k', 'n'], keep='last')
    else:
        df = df_new

    # Compute central states for all rows (backfills existing rows too)
    def compute_central(row):
        coset_func = cosets.get(row['coset'])
        if coset_func is None:
            return None
        central = coset_func(int(row['n']))
        return json.dumps(central) if central is not None else None
    
    df['central'] = df.apply(compute_central, axis=1)

    # Sort and save
    df = df.sort_values(['coset', 'k', 'n']).reset_index(drop=True)
    df.to_csv(csv_path, index=False)
    print(f"Saved: {csv_path} ({len(df)} total rows, {len(df_new)} new)")
    return df

def load_results(group_name):
    """Load results from CSV, parsing growth back to list."""
    group_dir = get_group_dir(group_name)
    df = pd.read_csv(f"{group_dir}/data.csv")
    df['growth'] = df['growth'].apply(json.loads)  # Parse JSON string back to list
    if 'central' in df.columns:
        df['central'] = df['central'].apply(lambda x: json.loads(x) if pd.notna(x) else None)
    return df

def plot_group_results(group_name, df):
    """Create interactive Plotly plots for diameter, growth, and last layer size."""
    group_dir = get_group_dir(group_name)
    coset_names = list(df['coset'].unique())
    k_values = sorted(df['k'].unique())

    # Pre-parse growth column
    df = df.copy()
    df['growth_parsed'] = df['growth'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)

    # ===== PLOT 1: Diameter vs n (with coset dropdown) =====
    fig1 = go.Figure()

    for coset_name in coset_names:
        coset_df = df[df['coset'] == coset_name]
        for k_val in k_values:
            k_df = coset_df[coset_df['k'] == k_val].sort_values('n')
            if len(k_df) > 0:
                fig1.add_trace(go.Scatter(
                    x=k_df['n'], y=k_df['diameter'],
                    mode='lines+markers', name=f'k={k_val}',
                    visible=(coset_name == coset_names[0]),
                    hovertemplate='n=%{x}<br>diameter=%{y}<br>k=' + str(k_val)
                ))

    # Add dropdown for coset selection with per-coset axis ranges
    buttons = []
    traces_per_coset = len(k_values)
    for i, coset_name in enumerate(coset_names):
        visibility = [False] * (len(coset_names) * traces_per_coset)
        for j in range(traces_per_coset):
            visibility[i * traces_per_coset + j] = True
        
        # Calculate ranges for this coset
        coset_df = df[df['coset'] == coset_name]
        n_range = [coset_df['n'].min() - 0.5, coset_df['n'].max() + 0.5]
        diameter_range = [0, coset_df['diameter'].max() * 1.05]
        
        buttons.append(dict(label=coset_name, method='update',
                           args=[{'visible': visibility},
                                 {'xaxis.range': n_range, 'yaxis.range': diameter_range}]))

    # Set initial ranges
    init_coset_df = df[df['coset'] == coset_names[0]]
    fig1.update_layout(
        title=dict(text=f'Diameter vs n - {group_name} (perm_type={PERM_TYPE})', y=0.95),
        xaxis_title='n', yaxis_title='Diameter',
        xaxis=dict(range=[init_coset_df['n'].min() - 0.5, init_coset_df['n'].max() + 0.5]),
        yaxis=dict(range=[0, init_coset_df['diameter'].max() * 1.05]),
        updatemenus=[dict(
            buttons=buttons, 
            direction='down', 
            x=0.0, 
            xanchor='left',
            y=1.02,
            yanchor='bottom',
            showactive=True
        )],
        legend=dict(x=1.02, y=1),
        height=650,
        margin=dict(t=100)
    )
    fig1.write_html(f'{group_dir}/diameter.html')
    fig1.show()

    # ===== PLOT 2: Growth curves (with coset + k dropdowns) =====
    fig2 = go.Figure()

    trace_map = {}  # (coset, k) -> list of trace indices
    trace_idx = 0
    for coset_name in coset_names:
        for k_val in k_values:
            trace_map[(coset_name, k_val)] = []
            coset_k_df = df[(df['coset'] == coset_name) & (df['k'] == k_val)].sort_values('n')
            for _, row in coset_k_df.iterrows():
                growth = row['growth_parsed']
                fig2.add_trace(go.Scatter(
                    x=list(range(len(growth))), y=growth,
                    mode='lines+markers', name=f"n={row['n']}",
                    visible=(coset_name == coset_names[0] and k_val == k_values[0]),
                    hovertemplate='distance=%{x}<br>layer_size=%{y}<br>n=' + str(row['n'])
                ))
                trace_map[(coset_name, k_val)].append(trace_idx)
                trace_idx += 1

    total_traces = trace_idx
    
    # Build combined buttons for (coset, k) pairs with per-selection axis ranges
    buttons_combined = []
    first_button = True
    for coset_name in coset_names:
        for k_val in k_values:
            if not trace_map.get((coset_name, k_val), []):
                continue
            visibility = [False] * total_traces
            for idx in trace_map[(coset_name, k_val)]:
                visibility[idx] = True
            
            # Calculate ranges for this (coset, k) combination
            coset_k_df = df[(df['coset'] == coset_name) & (df['k'] == k_val)]
            growths = coset_k_df['growth_parsed'].tolist()
            if growths:
                max_distance = max(len(g) for g in growths)
                max_layer = max(max(g) for g in growths)
                min_layer = min(min(g) for g in growths if min(g) > 0)
                distance_range = [-0.5, max_distance + 0.5]
                layer_range = [np.log10(min_layer * 0.5), np.log10(max_layer * 2)]
            else:
                distance_range = [0, 10]
                layer_range = [0, 6]
            
            buttons_combined.append(dict(
                label=f'{coset_name}, k={k_val}', 
                method='update',
                args=[{'visible': visibility},
                      {'xaxis.range': distance_range, 'yaxis.range': layer_range}]
            ))
            
            if first_button:
                init_distance_range = distance_range
                init_layer_range = layer_range
                first_button = False

    fig2.update_layout(
        title=dict(text=f'Growth Curves - {group_name} (perm_type={PERM_TYPE})', y=0.95),
        xaxis_title='Distance', yaxis_title='Layer Size',
        yaxis_type='log',
        xaxis=dict(range=init_distance_range),
        yaxis=dict(range=init_layer_range),
        updatemenus=[dict(
            buttons=buttons_combined, 
            direction='down', 
            x=0.0, 
            xanchor='left',
            y=1.02,
            yanchor='bottom',
            showactive=True
        )],
        legend=dict(x=1.02, y=1),
        height=650,
        margin=dict(t=100)
    )
    fig2.write_html(f'{group_dir}/growth.html')
    fig2.show()

    # ===== PLOT 3: Last layer size vs n (with coset dropdown) =====
    fig3 = go.Figure()

    for coset_name in coset_names:
        coset_df = df[df['coset'] == coset_name]
        for k_val in k_values:
            k_df = coset_df[coset_df['k'] == k_val].sort_values('n')
            if len(k_df) > 0:
                fig3.add_trace(go.Scatter(
                    x=k_df['n'], y=k_df['last_layer_size'],
                    mode='lines+markers', name=f'k={k_val}',
                    visible=(coset_name == coset_names[0]),
                    hovertemplate='n=%{x}<br>last_layer=%{y}<br>k=' + str(k_val)
                ))

    buttons = []
    for i, coset_name in enumerate(coset_names):
        visibility = [False] * (len(coset_names) * len(k_values))
        for j in range(len(k_values)):
            visibility[i * len(k_values) + j] = True
        
        # Calculate ranges for this coset
        coset_df = df[df['coset'] == coset_name]
        n_range = [coset_df['n'].min() - 0.5, coset_df['n'].max() + 0.5]
        ll_min = coset_df['last_layer_size'].min()
        ll_max = coset_df['last_layer_size'].max()
        last_layer_range = [np.log10(ll_min * 0.5), np.log10(ll_max * 2)]
        
        buttons.append(dict(label=coset_name, method='update',
                           args=[{'visible': visibility},
                                 {'xaxis.range': n_range, 'yaxis.range': last_layer_range}]))

    # Set initial ranges
    init_coset_df = df[df['coset'] == coset_names[0]]
    init_ll_min = init_coset_df['last_layer_size'].min()
    init_ll_max = init_coset_df['last_layer_size'].max()
    fig3.update_layout(
        title=dict(text=f'Last Layer Size vs n - {group_name} (perm_type={PERM_TYPE})', y=0.95),
        xaxis_title='n', yaxis_title='Last Layer Size',
        yaxis_type='log',
        xaxis=dict(range=[init_coset_df['n'].min() - 0.5, init_coset_df['n'].max() + 0.5]),
        yaxis=dict(range=[np.log10(init_ll_min * 0.5), np.log10(init_ll_max * 2)]),
        updatemenus=[dict(
            buttons=buttons, 
            direction='down', 
            x=0.0, 
            xanchor='left',
            y=1.02,
            yanchor='bottom',
            showactive=True
        )],
        legend=dict(x=1.02, y=1),
        height=650,
        margin=dict(t=100)
    )
    fig3.write_html(f'{group_dir}/lastlayer.html')
    fig3.show()
    
    print(f"Interactive plots saved to {group_dir}/")

print("Helper functions defined!")

Helper functions defined!


---
## Group 1: "different" (3 cosets)
- 2Different: [0, 1, 1, 1, ...]
- 3Different: [0, 1, 2, 2, ...]
- 4Different: [0, 1, 2, 3, 3, ...]

In [ ]:
%%time
group_name = "different"

# Customize ranges as needed:
# - min_n, max_n: range of n values to compute
# - k_range: tuple (k_min, k_max) or None for all valid k
# - Results are cached: only new (coset, k, n) combinations are computed
results = run_group(group_name, COSET_GROUPS[group_name],
                    min_n=4, max_n=30, k_range=None)
df = save_results(group_name, results, COSET_GROUPS[group_name], append=True)
plot_group_results(group_name, df)
print("\nSample data:")
display(df.head(15))

2Different: 100%|██████████| 810/810 [00:00<00:00, 1463.82it/s, k=29, n=30]


  2Different: Skipped 378 cached, computed 0 new


3Different: 100%|██████████| 810/810 [00:00<00:00, 1483.66it/s, k=29, n=30]


  3Different: Skipped 378 cached, computed 0 new


4Different: 100%|██████████| 810/810 [00:00<00:00, 1456.93it/s, k=29, n=30]


  4Different: Skipped 378 cached, computed 0 new
Completed different (0 new results)
Saved: results_perm2/different/data.csv (1134 total rows, 0 new)


---
## Group 2: "then" (4 cosets)
- Binary0then1: [0,0,...,1,1,...]
- 0then1then2: [0,...,1,...,2,...]
- 0then1then2then3: quarters
- 0then1then2then3then4: fifths

In [ ]:
%%time
group_name = "then"

# Customize ranges as needed:
results = run_group(group_name, COSET_GROUPS[group_name],
                    min_n=4, max_n=30, k_range=None)
df = save_results(group_name, results, COSET_GROUPS[group_name], append=True)
plot_group_results(group_name, df)
print("\nSample data:")
display(df.head(15))

---
## Group 3: "coincide" (5 cosets)
- 2Coincide: [..., x, x]
- 3Coincide: [..., x, x, x]
- 4Coincide, 5Coincide, 6Coincide

In [ ]:
%%time
group_name = "coincide"

# Customize ranges as needed:
results = run_group(group_name, COSET_GROUPS[group_name],
                    min_n=4, max_n=30, k_range=None)
df = save_results(group_name, results, COSET_GROUPS[group_name], append=True)
plot_group_results(group_name, df)
print("\nSample data:")
display(df.head(15))

---
## Group 4: "repeats" (4 cosets)
- Binary01Repeats: [0,1,0,1,...]
- Binary01Repeats_1: [0,1,0,1,...] with 1s at end
- 012Repeats: [0,1,2,0,1,2,...]
- 011Repeats: [0,1,1,0,1,1,...]

In [ ]:
%%time
group_name = "repeats"

# Customize ranges as needed:
results = run_group(group_name, COSET_GROUPS[group_name],
                    min_n=24, max_n=25, coset_filter = "Binary01Repeats", k_range=None)
df = save_results(group_name, results, COSET_GROUPS[group_name], append=True)
plot_group_results(group_name, df)
print("\nSample data:")
display(df.head(15))

In [ ]:
%%time
group_name = "repeats"

# Customize ranges as needed:
results = run_group(group_name, COSET_GROUPS[group_name],
                    min_n=4, max_n=30, k_range=None)
df = save_results(group_name, results, COSET_GROUPS[group_name], append=True)
plot_group_results(group_name, df)
print("\nSample data:")
display(df.head(15))

---
## Group 5: "full_graph" (1 coset, max_n=12)
- FullGraph: No central state, explores entire permutation space
- **Warning**: n! grows very fast, so we limit to max_n=12

In [ ]:
%%time
group_name = "full_graph"

# Customize ranges as needed (max_n=12 recommended for full graph due to n! growth):
results = run_group(group_name, COSET_GROUPS[group_name],
                    min_n=4, max_n=12, k_range=None)
df = save_results(group_name, results, COSET_GROUPS[group_name], append=True)
plot_group_results(group_name, df)
print("\nSample data:")
display(df.head(15))

---
## Summary: List all output files

In [ ]:
import glob

print("Output files generated:")
for f in sorted(glob.glob(f"{OUTPUT_DIR}/*")):
    size = os.path.getsize(f)
    print(f"  {f} ({size:,} bytes)")